In [1]:
from dotenv import load_dotenv
import json
import openai
import glob
from pathlib import Path
import os
import glob


In [2]:
load_dotenv()
kaggle_data_path_str = os.getenv('KAGGLE_DATASET_PATH')
kaggle_data_path = Path(kaggle_data_path_str)/'subset'

In [3]:
fnames = glob.glob(str(kaggle_data_path/'*.json'))

In [4]:
fnames

['/Users/rbiswas/doc/sebx/data/kaggle_financial_data/subset/news_0047188.json',
 '/Users/rbiswas/doc/sebx/data/kaggle_financial_data/subset/news_0014010.json',
 '/Users/rbiswas/doc/sebx/data/kaggle_financial_data/subset/news_0042670.json',
 '/Users/rbiswas/doc/sebx/data/kaggle_financial_data/subset/news_0038485.json',
 '/Users/rbiswas/doc/sebx/data/kaggle_financial_data/subset/news_0023363.json',
 '/Users/rbiswas/doc/sebx/data/kaggle_financial_data/subset/news_0043962.json',
 '/Users/rbiswas/doc/sebx/data/kaggle_financial_data/subset/news_0036862.json',
 '/Users/rbiswas/doc/sebx/data/kaggle_financial_data/subset/news_0000403.json',
 '/Users/rbiswas/doc/sebx/data/kaggle_financial_data/subset/news_0056263.json',
 '/Users/rbiswas/doc/sebx/data/kaggle_financial_data/subset/news_0032088.json',
 '/Users/rbiswas/doc/sebx/data/kaggle_financial_data/subset/news_0037770.json',
 '/Users/rbiswas/doc/sebx/data/kaggle_financial_data/subset/blogs_0040123.json',
 '/Users/rbiswas/doc/sebx/data/kaggle_f

In [5]:
def get_prompt(text):
    return f"Given the text surrounded by triple backticks, please return the same text with minimal modifications but replacing every mention of a name or organization by its full name (as available) and also replace the pronouns by the full names of the entity which they refer to. If the full name corresponding to a pronoun or a name is not available, then leave it as it is rather than making up an answer. ```{text}```"

In [6]:
def get_rephrased_text(text, client):
    prompt = get_prompt(text)
    response = client.chat.completions.create(model='gpt-4o',
                                                temperature=0.0, 
                                                top_p=0.3,           
                                                messages=[{ "role": "user","content":prompt}])

    rephrased_text = response.choices[0].message.content
    if response.choices[0].finish_reason == 'stop':
        return rephrased_text, True
    else:
        return rephrased_text, False


    

In [7]:
def write_rephrased_json(fname:str, rephrased_data_path): 
    """
    Given a json file, read the text and write the rephrased text back to the file
    """
    with open(fname, 'r') as f:
        data = json.load(f)
    text = data['text']
    try:
        rephrased_text, success = get_rephrased_text(text, openai)
    except:
        success = False

    if success:
        data['rephrased_text'] = rephrased_text
        with open(rephrased_data_path/fname.split('/')[-1], 'w') as f:
            json.dump(data, f)

    return success

In [8]:

bads = []
rephrased_data_path = kaggle_data_path/'rephrased_subset'

os.makedirs(str(kaggle_data_path/'rephrased_subset'), exist_ok=True)

client = openai.OpenAI(timeout=60)
for fname in fnames:
    success = write_rephrased_json(fname, rephrased_data_path=rephrased_data_path)
    if not success:
        bads.append(fname)

In [9]:
bads

['/Users/rbiswas/doc/sebx/data/kaggle_financial_data/subset/news_0043431.json']